# Tópico 9/10 - Filtros FIR/IIR/Notch (Rejeita Faixa)

## Alunos
* Alejo Perdomo Milar
* João Mário C. I. Lago

# Installs & Imports

In [ ]:
!pip install -q numpy matplotlib scipy

In [ ]:
import math
import json
from typing import Tuple, List, Dict, Optional
from dataclasses import dataclass

import numpy as np
import matplotlib.pyplot as plt

from scipy.io import wavfile
from scipy import signal

import IPython.display as ipd
from IPython.display import display, Markdown

# Utils

In [ ]:
@dataclass
class FilterAnalysisData:
    """Stores complete filter analysis data"""
    # Identification
    title: str

    # Frequency data
    freq_axis: np.ndarray
    mag_before_db: np.ndarray
    mag_after_db: np.ndarray
    phase_before_deg: np.ndarray
    phase_after_deg: np.ndarray

    # Key metrics
    sampling_freq: float
    notch_freq: Optional[float] = None
    notch_depth_db: Optional[float] = None
    bandwidth_3db: Optional[float] = None
    q_factor: Optional[float] = None
    passband_ripple_db: Optional[float] = None
    target_freq_gain_db: Optional[float] = None  # NOVO: Ganho na frequência target

    # Raw data for audio playback
    x_before: Optional[np.ndarray] = None
    x_after: Optional[np.ndarray] = None

    def __repr__(self):
        return (f"FilterAnalysisData('{self.title}', "
                f"f_notch={self.notch_freq:.1f}Hz, "
                f"BW={self.bandwidth_3db:.2f}Hz, "
                f"Q={self.q_factor:.1f}, "
                f"depth={self.notch_depth_db:.1f}dB)")

filters_analysis_data = {
}


def calculate_filter_metrics(freq_axis: np.ndarray, mag_db: np.ndarray,
                            target_notch_freq: Optional[float] = None,
                            search_range_hz: float = 10) -> Dict:
    """
    Calculate key filter metrics from frequency response

    Parameters:
    -----------
    freq_axis : np.ndarray
        Frequency axis in Hz
    mag_db : np.ndarray
        Magnitude response in dB
    target_notch_freq : float, optional
        Expected notch frequency in Hz. If provided, search within ±search_range_hz
    search_range_hz : float
        Search range around target frequency (default: 60 Hz)

    Returns:
    --------
    dict with keys: notch_freq, notch_depth_db, bandwidth_3db, q_factor, passband_ripple_db, target_freq_gain_db
    """

    # Find notch frequency (minimum magnitude)
    if target_notch_freq is not None:
        # Search only within ±search_range_hz of target frequency
        search_mask = (freq_axis >= target_notch_freq - search_range_hz) & \
                    (freq_axis <= target_notch_freq + search_range_hz)

        if np.any(search_mask):
            # Find minimum within the search range
            search_indices = np.where(search_mask)[0]
            local_min_idx = np.argmin(mag_db[search_mask])
            notch_idx = search_indices[local_min_idx]
        else:
            # Fallback: search entire spectrum
            notch_idx = np.argmin(mag_db)
    else:
        # No target specified, search entire spectrum
        notch_idx = np.argmin(mag_db)

    notch_freq = freq_axis[notch_idx]
    notch_depth = mag_db[notch_idx]

    # Find the magnitude at the target frequency
    target_freq_gain = None
    if target_notch_freq is not None:
        # Encontrar o índice mais próximo da frequência target
        target_idx = np.argmin(np.abs(freq_axis - target_notch_freq))
        target_freq_gain = mag_db[target_idx]

    # Calculate -3dB bandwidth
    target_magnitude = notch_depth + 3  # 3 dB above the notch

    # Search for -3dB points around the notch (within reasonable range)
    if target_notch_freq is not None:
        # Search within ±search_range_hz for bandwidth calculation
        bw_search_mask = (freq_axis >= target_notch_freq - search_range_hz) & \
                        (freq_axis <= target_notch_freq + search_range_hz)
        above_target = np.where((mag_db > target_magnitude) & bw_search_mask)[0]
    else:
        above_target = np.where(mag_db > target_magnitude)[0]

    bandwidth_3db = None
    q_factor = None

    if len(above_target) > 0:
        # Find frequencies on left and right of notch
        left_idx = above_target[above_target < notch_idx]
        right_idx = above_target[above_target > notch_idx]

        if len(left_idx) > 0 and len(right_idx) > 0:
            left_freq = freq_axis[left_idx[-1]]
            right_freq = freq_axis[right_idx[0]]
            bandwidth_3db = right_freq - left_freq
            q_factor = notch_freq / bandwidth_3db if bandwidth_3db > 0 else None

    # Calculate passband ripple (excluding notch region)
    if target_notch_freq is not None:
        # Exclude region within ±search_range_hz of target
        notch_exclusion = (freq_axis < target_notch_freq - search_range_hz) | \
                        (freq_axis > target_notch_freq + search_range_hz)
    else:
        # Exclude region within ±20% of detected notch freq
        notch_exclusion = (freq_axis < notch_freq * 0.8) | \
                        (freq_axis > notch_freq * 1.2)

    passband_mag = mag_db[notch_exclusion]
    passband_ripple = np.max(passband_mag) - np.min(passband_mag)

    return {
        'notch_freq': notch_freq,
        'notch_depth_db': notch_depth,
        'bandwidth_3db': bandwidth_3db,
        'q_factor': q_factor,
        'passband_ripple_db': passband_ripple,
        'target_freq_gain_db': target_freq_gain  # NOVO
    }

def linear_to_db(x: np.ndarray) -> np.ndarray:
    return 20 * np.log10(np.abs(x))

def plot_filter_fft(x_before, x_after, fs, title):
    plt.style.use('seaborn-v0_8-paper')
    # Create subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'FFT Analysis - {title}', fontsize=16)

    # Zero-padding for smoother curves
    pad_factor = 4

    # FFT before filter
    N_padded = len(x_before) * pad_factor
    f_before = np.fft.fft(x_before, n=N_padded)
    freq_axis = np.fft.fftfreq(N_padded, 1/fs)[:N_padded//2]

    # Magnitude before filter
    mag_before = np.abs(f_before)[:N_padded//2] / np.abs(f_before).max()
    ax1.plot(freq_axis, linear_to_db(mag_before),
            linewidth=1.5, color='#2E86AB', alpha=0.9)
    ax1.set_xlabel('Frequency (Hz)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Magnitude (dB)', fontsize=12, fontweight='bold')
    ax1.set_title('Magnitude Spectrum - Before Filter', fontsize=13, pad=10)
    ax1.set_xlim(20, 20000)
    ax1.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    ax1.minorticks_on()
    ax1.grid(True, which='minor', linestyle=':', linewidth=0.3, alpha=0.5)

    # Phase before filter
    phase_before = np.angle(f_before)[:N_padded//2]
    ax2.plot(freq_axis, np.degrees(phase_before),
            linewidth=1.5, color='#A23B72', alpha=0.9)
    ax2.set_xlabel('Frequency (Hz)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Phase (degrees)', fontsize=12, fontweight='bold')
    ax2.set_title('Phase Response - Before Filter', fontsize=13, pad=10)
    ax2.set_xlim(20, 20000)
    ax2.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    ax2.minorticks_on()
    ax2.grid(True, which='minor', linestyle=':', linewidth=0.3, alpha=0.5)
    ax2.set_yticks([-180, -90, 0, 90, 180])

    # FFT after filter
    N_padded = len(x_after) * pad_factor
    f_after = np.fft.fft(x_after, n=N_padded)
    freq_axis = np.fft.fftfreq(N_padded, 1/fs)[:N_padded//2]

    # Magnitude after filter
    mag_after = np.abs(f_after)[:N_padded//2] / np.abs(f_after).max()
    ax3.plot(freq_axis, linear_to_db(mag_after),
            linewidth=1.5, color='#18A558', alpha=0.9)
    ax3.set_xlabel('Frequency (Hz)', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Magnitude (dB)', fontsize=12, fontweight='bold')
    ax3.set_title('Magnitude Spectrum - After Filter', fontsize=13, pad=10)
    ax3.set_xlim(20, 20000)
    ax3.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    ax3.minorticks_on()
    ax3.grid(True, which='minor', linestyle=':', linewidth=0.3, alpha=0.5)

    # Phase after filter
    phase_after = np.angle(f_after)[:N_padded//2]
    ax4.plot(freq_axis, np.degrees(phase_after),
            linewidth=1.5, color='#F18F01', alpha=0.9)
    ax4.set_xlabel('Frequency (Hz)', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Phase (degrees)', fontsize=12, fontweight='bold')
    ax4.set_title('Phase Response - After Filter', fontsize=13, pad=10)
    ax4.set_xlim(20, 20000)
    ax4.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    ax4.minorticks_on()
    ax4.grid(True, which='minor', linestyle=':', linewidth=0.3, alpha=0.5)
    ax4.set_yticks([-180, -90, 0, 90, 180])

    plt.tight_layout()
    plt.show()

    # Calculate the filter metrics
    filter_metrics = calculate_filter_metrics(freq_axis, linear_to_db(mag_after), 875)

    # Save the filter analysis data
    filters_analysis_data[title] = FilterAnalysisData(
        title=title,
        freq_axis=freq_axis,
        mag_before_db=linear_to_db(mag_before),
        mag_after_db=linear_to_db(mag_after),
        phase_before_deg=np.degrees(phase_before),
        phase_after_deg=np.degrees(phase_after),
        sampling_freq=fs,
        **filter_metrics,
        x_before=x_before,
        x_after=x_after,
    )

# 1) Identificação do sinal espúrio

Verifique os sinais de voz contidos nos arquivos disponíveis no Moodle (individual para cada grupo ou estudante). Identifique o sinal espúrio (neste caso, um tom de frequência fixa) que mascara o sinal de interesse, usando OcenAudio ou Python.

## Espectrograma do sinal original

<div style="display: flex; justify-content: center;">
    <img src="./assets/audio-teste-ruido-G1-spectrum.png" alt="Espectrograma do sinal original" height="500">
</div>

## FFT do sinal original

<div style="display: flex; justify-content: center;">
    <img src="./assets/audio-teste-ruido-G1-tone-fft.png" alt="FFT Beep Tone Frequence" height="500">
</div>

Pode se observar que o sinal possui um beep constante na frequência de 875Hz.

# 2) Requisitos de projeto

A partir do sinal de voz, especifique os requisitos gerais de projeto de um filtro digital rejeita-faixa: frequências de corte inferior e superior, atenuação na banda de rejeição e frequência de amostragem.

## Parâmetros do filtro

Observando a FFT do sinal original, podemos identificar a frequência do tom como sendo 875Hz sendo que sua energia se estende até em torno de $850Hz$ até $900Hz$.
Além disso pode-se observar que o sinal no seu pico possui $-20dB$ e que a energia media ao redor é de $-70dB$, necessitando de uma atenuação de $-50dB$ na banda de rejeição minima.

* Frequência de amostragem: 48kHz
* Frequência do tom: 875Hz
* Faixa de rejeição: 850Hz a 900Hz
* Atenuação na banda de rejeição: -50dB (Mínimo)



# 3) Projeto de um filtro FIR

Utilize o Python (pyFDA ou scripts próprios) para projetar, simular e validar um filtro FIR obtido pelo método de janelamento para atenuar o sinal espúrio. Mostre os resultados no tempo e frequência usando um sinal sweep (ou ruído branco) e o sinal de voz corrompido.

## Parâmetros de design do filtro

<div style="display: flex; justify-content: center;">
    <img src="./assets/filters/fir/pyfdax_parameters.png" alt="FIR Specs" height="500">
</div>

## Resposta em frequência (Teórica)

#### Visão ampla

<div style="display: flex; justify-content: center;">
    <img src="./assets/filters/fir/teroic_response_20k.png" alt="FIR Bode Broad" height="500">
</div>

#### Visão estreita (Próximo do tom)

<div style="display: flex; justify-content: center;">
    <img src="./assets/filters/fir/teroic_response_0.5k_1.5k.png" alt="FIR Bode Narrow" height="500">
</div>

Pode ser observado que o vale mais a esquerda está alinhado com o os $870Hz$, o filtro não é perfeito pois para obter-se um filtro mais estreito seria necessário aumentar a ordem do filtro e aumentaria muito a complexidade computacional, assim não sendo viável para execução posterior na placa em real-time.


## Aplicando o filtro FIR (Python) no áudio original

In [ ]:
# Read audio file
fs, x = wavfile.read("./assets/audio-teste-ruido-G1.wav")

# Load FIR filter project from pyFDA json
with open("./assets/filters/fir/bandstop-fir.json", "r") as f:
    fir_project = json.load(f)
    # Extract b and a coefficients
    [b, a] = fir_project["ba"]
    # Extract sampling frequency
    fir_fs = int(fir_project["f_s_scale"] * fir_project["f_S"])

# Check FS matches
assert fs == fir_fs, "Loaded audio and FIR filter have different sampling frequencies"

# Print a markdown table with the b and a coefficients
table = "### FIR filter coefficients:\n\n"
table += "| Index | B | A |\n"
table += "|-------|---|---|\n"
for i, (b_i, a_i) in enumerate(zip(b, a)):
    table += f"| {i} | {b_i:.6g} | {a_i:.6g} |\n"

# Display
display(Markdown(table))

# Allow play original audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

y = signal.lfilter(b, a, x)
# Normalize Y
y = y / np.abs(y).max()

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "Audio File (Tone) FIR (Python)")

# Save wav normalized
wavfile.write("./generated/python/fir/audio-teste-ruido-G1-filtered.wav", fs, y)

## Teste do filtro FIR (Python) utilizando *ruído branco*

In [ ]:
# Generates a white noise signal and applies a FIR filter
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = np.random.randn(fs * 10)

y = signal.lfilter(b, a, x)

display(Markdown("### Original White Noise:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered White Noise:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis of White Noise:"))
plot_filter_fft(x, y, fs, "White Noise (Python)")

## Teste do filtro FIR (Python) utilizando *CHIRP* Sweep $0Hz-20kHz$

In [ ]:
# Sweep containing all frequencies from 20 Hz to 20 kHz
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = signal.chirp(t, 20, 10, 20000, method="linear")

y = signal.lfilter(b, a, x)

display(Markdown("### Original Chirp Signal:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered Chirp Signal:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis of Chirp Signal:"))
plot_filter_fft(x, y, fs, "Chirp Signal (Python)")

# 4) Implementação do filtro FIR em hardware
Modifique o projeto base do Cortex-M4 para implementar o filtro no kit de desenvolvimento em ponto flutuante. Compare com a sua implementação em computador (Python). Reimplemente o filtro no kit de desenvolvimento, agora em ponto fixo. Compare com a implementação em em ponto flutuante. Para todas as implementações (FIR, IIR e notch), obtenha os gráficos dos sinais no tempo e em frequência. Analise os resultados em relação ao desempenho do filtro e complexidade computacional. Indique possíveis aplicações, vantagens e desvantagens de cada implementação.

Utilizando o arquivo python disponibilizado pelo professor, [`Le_CSV_Pyfda_gera_coeff_FIR.py`](./scripts/Le_CSV_Pyfda_gera_coeff_FIR.py) foi realizado a conversão dos coeficientes no formato pyFDA para o formato CMSIS-DSP.

## `coeffs_FIR.h`

```c
#ifndef INCLUDE_COEFFS_FIR_H_
#define INCLUDE_COEFFS_FIR_H_

#include <stdint.h>

#include "filter.h"

#ifndef NUM_TAPS
#define NUM_TAPS 206
#endif

/*************************************/
/*     FIR Filter Coefficients       */
const float32_t firCoeffs32[206] = {  -1.215805e-04,  -9.500654e-05,  -6.515504e-05,  -3.301523e-05,  +3.079072e-07,  +3.355826e-05,  +6.529878e-05,  +9.389229e-05,  +1.174979e-04,  +1.340872e-04,  +1.414824e-04,  +1.374197e-04,  +1.196355e-04,  +8.597491e-05,  +3.451907e-05,  -3.627559e-05,  -1.274320e-04,  -2.393096e-04,  -3.714738e-04,  -5.225857e-04,  -6.903236e-04,  -8.713425e-04,  -1.061282e-03,  -1.254823e-03,  -1.445804e-03,  -1.627391e-03,  -1.792300e-03,  -1.933070e-03,  -2.042373e-03,  -2.113355e-03,  -2.139991e-03,  -2.117433e-03,  -2.042349e-03,  -1.913216e-03,  -1.730562e-03,  -1.497136e-03,  -1.217995e-03,  -9.004843e-04,  -5.541234e-04,  -1.903692e-04,  +1.777219e-04,  +5.359399e-04,  +8.694586e-04,  +1.163450e-03,  +1.403753e-03,  +1.577577e-03,  +1.674202e-03,  +1.685654e-03,  +1.607322e-03,  +1.438483e-03,  +1.182700e-03,  +8.480834e-04,  +4.473769e-04,  -2.145615e-06,  -4.789758e-04,  -9.578674e-04,  -1.410528e-03,  -1.806489e-03,  -2.114138e-03,  -2.301878e-03,  -2.339379e-03,  -2.198883e-03,  -1.856509e-03,  -1.293514e-03,  -4.974530e-04,  +5.368038e-04,  +1.806253e-03,  +3.299177e-03,  +4.994833e-03,  +6.863449e-03,  +8.866530e-03,  +1.095750e-02,  +1.308264e-02,  +1.518239e-02,  +1.719282e-02,  +1.904745e-02,  +2.067917e-02,  +2.202235e-02,  +2.301492e-02,  +2.360059e-02,  +2.373079e-02,  +2.336664e-02,  +2.248059e-02,  +2.105779e-02,  +1.909716e-02,  +1.661197e-02,  +1.363015e-02,  +1.019400e-02,  +6.359567e-03,  +2.195492e-03,  -2.218472e-03,  -6.793384e-03,  -1.143334e-02,  -1.603794e-02,  -2.050499e-02,  -2.473328e-02,  -2.862532e-02,  -3.209015e-02,  -3.504588e-02,  -3.742201e-02,  -3.916148e-02,  -4.022230e-02,  +9.536949e-01,  -4.022230e-02,  -3.916148e-02,  -3.742201e-02,  -3.504588e-02,  -3.209015e-02,  -2.862532e-02,  -2.473328e-02,  -2.050499e-02,  -1.603794e-02,  -1.143334e-02,  -6.793384e-03,  -2.218472e-03,  +2.195492e-03,  +6.359567e-03,  +1.019400e-02,  +1.363015e-02,  +1.661197e-02,  +1.909716e-02,  +2.105779e-02,  +2.248059e-02,  +2.336664e-02,  +2.373079e-02,  +2.360059e-02,  +2.301492e-02,  +2.202235e-02,  +2.067917e-02,  +1.904745e-02,  +1.719282e-02,  +1.518239e-02,  +1.308264e-02,  +1.095750e-02,  +8.866530e-03,  +6.863449e-03,  +4.994833e-03,  +3.299177e-03,  +1.806253e-03,  +5.368038e-04,  -4.974530e-04,  -1.293514e-03,  -1.856509e-03,  -2.198883e-03,  -2.339379e-03,  -2.301878e-03,  -2.114138e-03,  -1.806489e-03,  -1.410528e-03,  -9.578674e-04,  -4.789758e-04,  -2.145615e-06,  +4.473769e-04,  +8.480834e-04,  +1.182700e-03,  +1.438483e-03,  +1.607322e-03,  +1.685654e-03,  +1.674202e-03,  +1.577577e-03,  +1.403753e-03,  +1.163450e-03,  +8.694586e-04,  +5.359399e-04,  +1.777219e-04,  -1.903692e-04,  -5.541234e-04,  -9.004843e-04,  -1.217995e-03,  -1.497136e-03,  -1.730562e-03,  -1.913216e-03,  -2.042349e-03,  -2.117433e-03,  -2.139991e-03,  -2.113355e-03,  -2.042373e-03,  -1.933070e-03,  -1.792300e-03,  -1.627391e-03,  -1.445804e-03,  -1.254823e-03,  -1.061282e-03,  -8.713425e-04,  -6.903236e-04,  -5.225857e-04,  -3.714738e-04,  -2.393096e-04,  -1.274320e-04,  -3.627559e-05,  +3.451907e-05,  +8.597491e-05,  +1.196355e-04,  +1.374197e-04,  +1.414824e-04,  +1.340872e-04,  +1.174979e-04,  +9.389229e-05,  +6.529878e-05,  +3.355826e-05,  +3.079072e-07,  -3.301523e-05,  -6.515504e-05,  -9.500654e-05,  -1.215805e-04,  +0.000000e+00 };

const q15_t firCoeffsQ15[206] = {  -4,  -3,  -2,  -1,  0,  1,  2,  3,  4,  4,  5,  5,  4,  3,  1,  -1,  -4,  -8,  -12,  -17,  -23,  -29,  -35,  -41,  -47,  -53,  -59,  -63,  -67,  -69,  -70,  -69,  -67,  -63,  -57,  -49,  -40,  -30,  -18,  -6,  6,  18,  28,  38,  46,  52,  55,  55,  53,  47,  39,  28,  15,  0,  -16,  -31,  -46,  -59,  -69,  -75,  -77,  -72,  -61,  -42,  -16,  18,  59,  108,  164,  225,  291,  359,  429,  497,  563,  624,  678,  722,  754,  773,  778,  766,  737,  690,  626,  544,  447,  334,  208,  72,  -73,  -223,  -375,  -526,  -672,  -810,  -938,  -1052,  -1148,  -1226,  -1283,  -1318,  31251,  -1318,  -1283,  -1226,  -1148,  -1052,  -938,  -810,  -672,  -526,  -375,  -223,  -73,  72,  208,  334,  447,  544,  626,  690,  737,  766,  778,  773,  754,  722,  678,  624,  563,  497,  429,  359,  291,  225,  164,  108,  59,  18,  -16,  -42,  -61,  -72,  -77,  -75,  -69,  -59,  -46,  -31,  -16,  0,  15,  28,  39,  47,  53,  55,  55,  52,  46,  38,  28,  18,  6,  -6,  -18,  -30,  -40,  -49,  -57,  -63,  -67,  -69,  -70,  -69,  -67,  -63,  -59,  -53,  -47,  -41,  -35,  -29,  -23,  -17,  -12,  -8,  -4,  -1,  1,  3,  4,  5,  5,  4,  4,  3,  2,  1,  0,  -1,  -2,  -3,  -4,  0 };
FilterTypeDef filterType=FIR_Q15;
/***********************************/

#endif /* INCLUDE_COEFFS_FIR_H_ */
```

## Áudio filtrado utilizando o kit de desenvolvimento com STM32 (Float 32)

In [ ]:
# Read audio file
fs, x = wavfile.read("./assets/audio-teste-ruido-G1.wav")
fs, y = wavfile.read("./generated/kit/fir/f32/audio-teste-ruido-G1-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "Audio File (Tone) FIR (Float 32 KIT)")

## *Ruído Branco* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32)

In [ ]:
# Generates a white noise
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = np.random.randn(fs * 10)

# Read audio file
fs, y = wavfile.read("./generated/kit/fir/f32/white-noise-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "White Noise (Float 32 KIT)")

## *CHIRP* Sweep $0Hz-20kHz$ filtrado utilizando o kit de desenvolvimento com STM32 (Float 32)

In [ ]:
# Sweep containing all frequencies from 20 Hz to 20 kHz
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = signal.chirp(t, 20, 10, 20000, method="linear")

# Read audio file
fs, y = wavfile.read("./generated/kit/fir/f32/chirp-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "Chirp Signal FIR (Float 32 KIT)")

## Áudio filtrado utilizando o kit de desenvolvimento com STM32 (Q15)

In [ ]:
# Read audio file
fs, x = wavfile.read("./assets/audio-teste-ruido-G1.wav")
fs, y = wavfile.read("./generated/kit/fir/q15/audio-teste-ruido-G1-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "Audio File (Tone) FIR (Q15 KIT)")

## *Ruído Branco* filtrado utilizando o kit de desenvolvimento com STM32 (Q15)

In [ ]:
# Generates a white noise
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = np.random.randn(fs * 10)

# Read audio file
fs, y = wavfile.read("./generated/kit/fir/q15/white-noise-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "White Noise (Q15 KIT))")

## *CHIRP* Sweep $0Hz-20kHz$ filtrado utilizando o kit de desenvolvimento com STM32 (Q15)

In [ ]:
# Sweep containing all frequencies from 20 Hz to 20 kHz
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = signal.chirp(t, 20, 10, 20000, method="linear")

# Read audio file
fs, y = wavfile.read("./generated/kit/fir/q15/chirp-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "Chirp Signal FIR (Q15 KIT))")

# 5) Projeto de um filtro IIR
Utilize o Python (pyFDA ou scripts próprios) para projetar, simular e validar um filtro IIR obtido pelo método das aproximações clássicas para atenuar o sinal espúrio. Mostre os resultados no tempo e frequência usando um sinal sweep (ou ruído branco) e o sinal de voz corrompido.

## Parâmetros de design do filtro

<div style="display: flex; justify-content: center;">
    <img src="./assets/filters/iir/pyfdax_parameters.png" alt="IIR Specs" height="500">
</div>

## Resposta em frequência (Teórica)

#### Visão ampla

<div style="display: flex; justify-content: center;">
    <img src="./assets/filters/iir/teroic_response_20k.png" alt="IIR Bode Broad" height="500">
</div>

#### Visão estreita (Próximo do tom)

<div style="display: flex; justify-content: center;">
    <img src="./assets/filters/iir/teroic_response_0.5k_1.5k.png" alt="IIR Bode Narrow" height="500">
</div>

Pode ser observado que com uma ordem muito menor (4), o filtro IIR consegue atingir uma atenuação muito estreita e boa na região onde o tom está presente.

## Aplicando o filtro IIR (Python) no áudio original

In [ ]:
# Read audio file
fs, x = wavfile.read("./assets/audio-teste-ruido-G1.wav")

# Load IIR filter project from pyFDA json
with open("./assets/filters/iir/bandstop-iir.json", "r") as f:
    iir_project = json.load(f)
    # Extract b and a coefficients
    [b, a] = iir_project["ba"]
    # Extract sampling frequency
    iir_fs = int(iir_project["f_s_scale"] * iir_project["f_S"])

# Check FS matches
assert fs == iir_fs, "Loaded audio and IIR filter have different sampling frequencies"

# Print a markdown table with the b and a coefficients
table = "### IIR filter coefficients:\n\n"
table += "| Index | B | A |\n"
table += "|-------|---|---|\n"
for i, (b_i, a_i) in enumerate(zip(b, a)):
    table += f"| {i} | {b_i:.6g} | {a_i:.6g} |\n"

# Display
display(Markdown(table))

# Allow play original audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

y = signal.lfilter(b, a, x)
# Normalize Y
y = y / np.abs(y).max()

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "Audio File (Tone) IIR (Python)")

# Save wav normalized
wavfile.write("./generated/python/iir/audio-teste-ruido-G1-filtered.wav", fs, y)

## Teste do filtro IIR (Python) utilizando *ruído branco*

In [ ]:
# Generates a white noise signal and applies a IIR filter
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = np.random.randn(fs * 10)

y = signal.lfilter(b, a, x)

display(Markdown("### Original White Noise:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered White Noise:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis of White Noise:"))
plot_filter_fft(x, y, fs, "White Noise (Python)")

## Teste do filtro IIR (Python) utilizando *CHIRP* Sweep $0Hz-20kHz$

In [ ]:
# Sweep containing all frequencies from 20 Hz to 20 kHz
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = signal.chirp(t, 20, 10, 20000, method="linear")

y = signal.lfilter(b, a, x)

display(Markdown("### Original Chirp Signal:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered Chirp Signal:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis of Chirp Signal:"))
plot_filter_fft(x, y, fs, "Chirp Signal (Python)")

# 6) Implementação do filtro IIR em hardware
Modifique o projeto base do Cortex-M4 para implementar o filtro no kit de desenvolvimento em ponto flutuante (cascata de estágios de 2a ordem). Analise os resultados.

Utilizando o arquivo python disponibilizado pelo professor, [`Le_CSV_CMSIS_SOS_Pyfda_gera_coeff_IIR.py`](./scripts/Le_CSV_CMSIS_SOS_Pyfda_gera_coeff_IIR.py) foi realizado a conversão dos coeficientes no formato pyFDA para o formato CMSIS-DSP.

## `coeffs_IIR.h`

```c
#ifndef INCLUDE_COEFFS_IIR_H_
#define INCLUDE_COEFFS_IIR_H_

#include <stdint.h>

#ifndef NUM_STAGES
#define NUM_STAGES 4
#endif

/*********************************************************/
/*                     IIR SOS Filter Coefficients       */
const float32_t iirCoeffsF32[20] = { // b0,b1,b2,a1,a2,... by stage
    +9.432219266891e-01, -1.873967409134e+00, +9.434114694595e-01,
    +1.962563157082e+00, -9.778336286545e-01,
    +1.000000000000e+00, -1.987024545670e+00, +9.997857809067e-01,
    +1.970170974731e+00, -9.811660647392e-01,
    +1.000000000000e+00, -1.986106395721e+00, +9.999063611031e-01,
    +1.981383442879e+00, -9.966115355492e-01,
    +1.000000000000e+00, -1.987682700157e+00, +1.000107049942e+00,
    +1.985867977142e+00, -9.970877766609e-01
};
/*********************************************************/

#endif /* INCLUDE_COEFFS_IIR_H_ */
```

## Áudio filtrado utilizando o kit de desenvolvimento com STM32 (Float 32)

In [ ]:
# Read audio file
fs, x = wavfile.read("./assets/audio-teste-ruido-G1.wav")
fs, y = wavfile.read("./generated/kit/iir/f32/audio-teste-ruido-G1-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "Audio File (Tone) IIR (Float 32 KIT)")

## *Ruído Branco* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32)

In [ ]:
# Generates a white noise
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = np.random.randn(fs * 10)

# Read audio file
fs, y = wavfile.read("./generated/kit/iir/f32/white-noise-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "White Noise IIR (Float 32 KIT)")

## *CHIRP* Sweep $0Hz-20kHz$ filtrado utilizando o kit de desenvolvimento com STM32 (Float 32)

In [ ]:
# Sweep containing all frequencies from 20 Hz to 20 kHz
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = signal.chirp(t, 20, 10, 20000, method="linear")

# Read audio file
fs, y = wavfile.read("./generated/kit/iir/f32/chirp-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "Chirp Signal IIR (Float 32 KIT)")

# 7) Projeto de um filtro notch

Projete um filtro notch (com realimentação, não o notch clássico) usando a estrutura apresentada em [1]. Avalie diferentes valores de realimentação. Compare os resultados com a aproximação clássica IIR implementada anteriormente.

### Artigo de Referência [1]:
* S. -C. Pei, B. -Y. Guo and W. -Y. Lu, "Narrowband Notch Filter Using Feedback Structure Tips & Tricks," in IEEE Signal Processing Magazine, vol. 33, no. 3, pp. 115-118, May 2016, doi: 10.1109/MSP.2016.2531578.

### Cálculo do filtro Notch:

$$
H_{n}(z) = 
\frac{(1+\alpha)\left(1 - 2\cos(\omega^{*})z^{-1} + z^{-2}\right)}
{(1+\alpha) - 2(\alpha+\rho)\cos(\omega^{*})z^{-1} + (\rho^{2}+\alpha)z^{-2}}
$$




In [ ]:
def calculate_notch_coefficients(f0: float, fs: float, rho: float = 0.99, alpha: float = 0.0) -> Tuple[List[float], List[float]]:
    """
    Calculate notch filter coefficients using Pei et al. method.

    Args:
        f0: Notch frequency in Hz
        fs: Sampling frequency in Hz
        rho: Pole radius (controls notch width, closer to 1 = narrower)
        alpha: Feedback parameter (0 for classical notch)

    Returns:
        Tuple of (numerator_coeffs, denominator_coeffs)
    """
    # Calculate normalized frequency
    omega_star = 2 * math.pi * f0 / fs
    cos_omega = math.cos(omega_star)

    # Calculate coefficients
    # Numerator: b = [1, -2*cos(omega*), 1]
    b0 = 1.0
    b1 = -2 * cos_omega
    b2 = 1.0

    # Denominator: a = [1, -2*(alpha+rho)/(1+alpha)*cos(omega*), (rho^2+alpha)/(1+alpha)]
    # Normalized so a0 = 1
    a0 = 1.0
    a1 = -(2 * (alpha + rho) / (1 + alpha)) * cos_omega
    a2 = (rho**2 + alpha) / (1 + alpha)

    return [b0, b1, b2], [a0, -a1, -a2]

def print_cmsis_dsp_format(b: List[float], a: List[float], f0: float, fs: float, rho: float, alpha: float):
    """Print coefficients in CMSIS-DSP format."""
    print(print_cmsis_dsp_format_str(b, a, f0, fs, rho, alpha))

def print_cmsis_dsp_format_str(b: List[float], a: List[float], f0: float, fs: float, rho: float, alpha: float):
    """Print coefficients in CMSIS-DSP format."""
    return f"""
  === CMSIS-DSP Format (SOS: {{b0,b1,b2,a1,a2}}) ===
  // Generated by notch_filter_calculator.py
  // f0 = {f0} Hz, fs = {fs} Hz, ρ = {rho}, α = {alpha}
  #ifndef INCLUDE_COEFFS_IIR_H_
  #define INCLUDE_COEFFS_IIR_H_

  #include <stdint.h>

  #ifndef NUM_STAGES
  #define NUM_STAGES 1
  #endif

  static const float32_t iirCoeffsF32[NUM_STAGES*5] = {{
      {b[0]:+.16e}f,
      {b[1]:+.16e}f,
      {b[2]:+.16e}f,
      {a[1]:+.16e}f,
      {a[2]:+.16e}f
  }};

  #endif /* INCLUDE_COEFFS_IIR_H_ */
  """

def print_python_format(b: List[float], a: List[float], f0: float, fs: float, rho: float, alpha: float):
    """Print coefficients in Python format."""
    print(f"\n=== Python Format ===")
    print(f"# f0 = {f0} Hz, fs = {fs} Hz, ρ = {rho}, α = {alpha}")
    print(f"b = np.array([{b[0]:.16f}, {b[1]:.16f}, {b[2]:.16f}])")
    print(f"a = np.array([{a[0]:.16f}, {a[1]:.16f}, {a[2]:.16f}])")

## Teste do filtro Notch rho 0.9 e alpha 0.3 (Python) no áudio original

In [ ]:
# Read audio file
fs, x = wavfile.read("./assets/audio-teste-ruido-G1.wav")

# Calculate the coefficients
[b0, b1, b2], [a0, a1, a2] = calculate_notch_coefficients(f0=874, fs=48000, rho=0.9, alpha=0.3)

b = [b0, b1, b2]
a = [a0, -a1, -a2]

# Print a markdown table with the b and a coefficients
table = "### Notch filter coefficients:\n\n"
table += "| Index | B | A |\n"
table += "|-------|---|---|\n"
for i, (b_i, a_i) in enumerate(zip(b, a)):
    table += f"| {i} | {b_i:.6g} | {a_i:.6g} |\n"

# Display
display(Markdown(table))

# Allow play original audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

y = signal.lfilter(b, a, x)
# Normalize Y
y = y / np.abs(y).max()

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "Audio File (Tone) Notch (Python)")

# Save wav normalized
wavfile.write("./generated/python/notch/audio-teste-ruido-G1-filtered.wav", fs, y)

## Teste do filtro Notch rho 0.9 e alpha 0.3 (Python) utilizando *ruído branco*

In [ ]:
# Generates a white noise signal and applies a FIR filter
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = np.random.randn(fs * 10)

y = signal.lfilter(b, a, x)

display(Markdown("### Original White Noise:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered White Noise:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis of White Noise:"))
plot_filter_fft(x, y, fs, "White Noise Notch (Python)")

## Teste do filtro Notch rho 0.9 e alpha 0.3 (Python) utilizando *CHIRP* Sweep $0Hz-20kHz$

In [ ]:
# Sweep containing all frequencies from 20 Hz to 20 kHz
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = signal.chirp(t, 20, 10, 20000, method="linear")

y = signal.lfilter(b, a, x)

display(Markdown("### Original Chirp Signal:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered Chirp Signal:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis of Chirp Signal:"))
plot_filter_fft(x, y, fs, "Chirp Signal Notch (Python)")

# 8) Implementação do filtro notch
Modifique o projeto base do Cortex-M4 para implementar o filtro notch no kit de desenvolvimento em ponto flutuante. Analise os resultados.

Entregue os resultados com todo o projeto do filtro (especificações de projeto e coeficientes gerados), o código C modificado, resultados e análise.

### Implementação

Realizamos testes com diferentes valores de ρ e α para encontrar o melhor equilíbrio. A escolha de ρ=0.9 e α=0.3 oferece um bom equilíbrio entre a profundidade do notch e a rejeição do ruído. O notch é suficientemente estreito para eliminar o tom de ruído, mas não tão estreito que fique impossível acertar a frequência exata. Isso é importante porque existem erros numéricos que causam uma variação de alguns Hz entre a frequência projetada e a prática, e se o notch fosse muito estreito, essa variação faria com que o filtro não rejeitasse adequadamente o tom.

In [ ]:
# Calculate the coefficients
[b0, b1, b2], [a0, a1, a2] = calculate_notch_coefficients(f0=874, fs=48000, rho=0.9, alpha=0.3)

b = [b0, b1, b2]
a = [a0, -a1, -a2]

# Print a markdown table with the b and a coefficients
table = "### Notch filter coefficients:\n\n"
table += "| Index | B | A |\n"
table += "|-------|---|---|\n"
for i, (b_i, a_i) in enumerate(zip(b, a)):
    table += f"| {i} | {b_i:.6g} | {a_i:.6g} |\n"

# Print the table
display(Markdown(table))

display(Markdown("### C code for the notch filter coefficientss"))
display(Markdown("```c" + print_cmsis_dsp_format_str(b, a, f0=874, fs=48000, rho=0.9, alpha=0.3) + "```"))


## Áudio filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.9 e alpha 0.3

In [ ]:
# Read audio file
fs, x = wavfile.read("./assets/audio-teste-ruido-G1.wav")
fs, y = wavfile.read("./generated/kit/notch/f32/rho_0.9_alpha_0.3/audio-teste-ruido-G1-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "Audio File (Tone) Notch (rho 0.9, alpha 0.3) (Float 32 KIT)")

## *Ruído Branco* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.9 e alpha 0.3

In [ ]:
# Generates a white noise
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = np.random.randn(fs * 10)

# Read audio file
fs, y = wavfile.read("./generated/kit/notch/f32/rho_0.9_alpha_0.3/white-noise-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "White Noise Notch (rho 0.9, alpha 0.3) (Float 32 KIT)")

## *CHIRP* Sweep $0Hz-20kHz$ filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.9 e alpha 0.3

In [ ]:
# Sweep containing all frequencies from 20 Hz to 20 kHz
fs = 48000
t = np.linspace(0, 10, fs * 10)
x = signal.chirp(t, 20, 10, 20000, method="linear")

# Read audio file
fs, y = wavfile.read("./generated/kit/notch/f32/rho_0.9_alpha_0.3/chirp-filtered.wav")

# Allow play filtered audio
display(Markdown("### Original audio:"))
display(ipd.Audio(x, rate=fs))

# Allow play filtered audio
display(Markdown("### Filtered audio:"))
display(ipd.Audio(y, rate=fs))

# Plot FFT analysis using the new function
display(Markdown("### FFT analysis:"))
plot_filter_fft(x, y, fs, "Chirp Signal Notch (rho 0.9, alpha 0.3) (Float 32 KIT)")

## Áudio filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.7 e alpha variavel

In [ ]:
for alpha in [0, 5, 10, 20]:

    display(Markdown(f"### Áudio filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.7 e alpha {alpha}"))
    # Read audio file
    fs, x = wavfile.read("./assets/audio-teste-ruido-G1.wav")
    fs, y = wavfile.read(f"./generated/kit/notch/f32/rho_0.7/alpha_{alpha}/audio-teste-ruido-G1-filtered.wav")

    # Allow play filtered audio
    display(Markdown("#### Original audio:"))
    display(ipd.Audio(x, rate=fs))

    # Allow play filtered audio
    display(Markdown("#### Filtered audio:"))
    display(ipd.Audio(y, rate=fs))

    # Plot FFT analysis using the new function
    display(Markdown("#### FFT analysis:"))
    plot_filter_fft(x, y, fs, f"Audio File (Tone) Notch (rho 0.7, alpha {alpha}) (Float 32 KIT)")

Como e possivel observar nos graficos acima, o aumento do alpha deixa o filtro mais estreito, mas tambem pode ficar tão estreito que não consegue remover todo o espectro do tom de ruido.

## *Ruído Branco* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.7 e alpha variavel

In [ ]:
for alpha in [0, 5, 10, 20]:

    display(Markdown(f"### *Ruído Branco* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.7 e alpha {alpha}"))
    # Generates a white noise
    fs = 48000
    t = np.linspace(0, 10, fs * 10)
    x = np.random.randn(fs * 10)

    # Read audio file
    fs, y = wavfile.read(f"./generated/kit/notch/f32/rho_0.7/alpha_{alpha}/white-noise-filtered.wav")

    # Allow play filtered audio
    display(Markdown("### Original audio:"))
    display(ipd.Audio(x, rate=fs))

    # Allow play filtered audio
    display(Markdown("### Filtered audio:"))
    display(ipd.Audio(y, rate=fs))

    # Plot FFT analysis using the new function
    display(Markdown("### FFT analysis:"))
    plot_filter_fft(x, y, fs, f"White Noise Notch (rho 0.7, alpha {alpha}) (Float 32 KIT)")

## *CHIRP* Sweep $0Hz-20kHz$ filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.7 e alpha variavel

In [ ]:
for alpha in [0, 5, 10, 20]:

    display(Markdown(f"### *CHIRP* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.7 e alpha {alpha}"))
    # Sweep containing all frequencies from 0 Hz to 20 kHz
    fs = 48000
    t = np.linspace(0, 10, fs * 10)
    x = signal.chirp(t, 0, 10, 20000, method="linear")

    # Read audio file
    fs, y = wavfile.read(f"./generated/kit/notch/f32/rho_0.7/alpha_{alpha}/chirp-filtered.wav")

    # Allow play filtered audio
    display(Markdown("### Original audio:"))
    display(ipd.Audio(x, rate=fs))

    # Allow play filtered audio
    display(Markdown("### Filtered audio:"))
    display(ipd.Audio(y, rate=fs))

    # Plot FFT analysis using the new function
    display(Markdown("### FFT analysis:"))
    plot_filter_fft(x, y, fs, f"*CHIRP* Notch (rho 0.7, alpha {alpha}) (Float 32 KIT)")

## Áudio filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.8 e alpha variavel

In [ ]:
for alpha in [0, 5, 10, 20]:

    display(Markdown(f"### Áudio filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.8 e alpha {alpha}"))
    # Read audio file
    fs, x = wavfile.read("./assets/audio-teste-ruido-G1.wav")
    fs, y = wavfile.read(f"./generated/kit/notch/f32/rho_0.8/alpha_{alpha}/audio-teste-ruido-G1-filtered.wav")

    # Allow play filtered audio
    display(Markdown("#### Original audio:"))
    display(ipd.Audio(x, rate=fs))

    # Allow play filtered audio
    display(Markdown("#### Filtered audio:"))
    display(ipd.Audio(y, rate=fs))

    # Plot FFT analysis using the new function
    display(Markdown("#### FFT analysis:"))
    plot_filter_fft(x, y, fs, f"Audio File (Tone) Notch (rho 0.8, alpha {alpha}) (Float 32 KIT)")

## *Ruído Branco* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.8 e alpha variavel

In [ ]:
for alpha in [0, 5, 10, 20]:

    display(Markdown(f"### *Ruído Branco* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.8 e alpha {alpha}"))
    # Generates a white noise
    fs = 48000
    t = np.linspace(0, 10, fs * 10)
    x = np.random.randn(fs * 10)

    # Read audio file
    fs, y = wavfile.read(f"./generated/kit/notch/f32/rho_0.8/alpha_{alpha}/white-noise-filtered.wav")

    # Allow play filtered audio
    display(Markdown("### Original audio:"))
    display(ipd.Audio(x, rate=fs))

    # Allow play filtered audio
    display(Markdown("### Filtered audio:"))
    display(ipd.Audio(y, rate=fs))

    # Plot FFT analysis using the new function
    display(Markdown("### FFT analysis:"))
    plot_filter_fft(x, y, fs, f"White Noise Notch (rho 0.8, alpha {alpha}) (Float 32 KIT)")

## *CHIRP* Sweep $0Hz-20kHz$ filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.8 e alpha variavel

In [ ]:
for alpha in [0, 5, 10, 20]:

    display(Markdown(f"### *CHIRP* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.8 e alpha {alpha}"))
    # Sweep containing all frequencies from 0 Hz to 20 kHz
    fs = 48000
    t = np.linspace(0, 10, fs * 10)
    x = signal.chirp(t, 0, 10, 20000, method="linear")

    # Read audio file
    fs, y = wavfile.read(f"./generated/kit/notch/f32/rho_0.8/alpha_{alpha}/chirp-filtered.wav")

    # Allow play filtered audio
    display(Markdown("### Original audio:"))
    display(ipd.Audio(x, rate=fs))

    # Allow play filtered audio
    display(Markdown("### Filtered audio:"))
    display(ipd.Audio(y, rate=fs))

    # Plot FFT analysis using the new function
    display(Markdown("### FFT analysis:"))
    plot_filter_fft(x, y, fs, f"*CHIRP* Notch (rho 0.8, alpha {alpha}) (Float 32 KIT)")

## Áudio filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.9 e alpha variavels

In [ ]:
for alpha in [0, 5, 10, 20]:

    display(Markdown(f"### Áudio filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.9 e alpha {alpha}"))
    # Read audio file
    fs, x = wavfile.read("./assets/audio-teste-ruido-G1.wav")
    fs, y = wavfile.read(f"./generated/kit/notch/f32/rho_0.9/alpha_{alpha}/audio-teste-ruido-G1-filtered.wav")

    # Allow play filtered audio
    display(Markdown("#### Original audio:"))
    display(ipd.Audio(x, rate=fs))

    # Allow play filtered audio
    display(Markdown("#### Filtered audio:"))
    display(ipd.Audio(y, rate=fs))

    # Plot FFT analysis using the new function
    display(Markdown("#### FFT analysis:"))
    plot_filter_fft(x, y, fs, f"Audio File (Tone) Notch (rho 0.9, alpha {alpha}) (Float 32 KIT)")

## *Ruído Branco* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.9 e alpha variavel

In [ ]:
for alpha in [0, 5, 10, 20]:

    display(Markdown(f"### *Ruído Branco* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.9 e alpha {alpha}"))
    # Generates a white noise
    fs = 48000
    t = np.linspace(0, 10, fs * 10)
    x = np.random.randn(fs * 10)

    # Read audio file
    fs, y = wavfile.read(f"./generated/kit/notch/f32/rho_0.9/alpha_{alpha}/white-noise-filtered.wav")

    # Allow play filtered audio
    display(Markdown("### Original audio:"))
    display(ipd.Audio(x, rate=fs))

    # Allow play filtered audio
    display(Markdown("### Filtered audio:"))
    display(ipd.Audio(y, rate=fs))

    # Plot FFT analysis using the new function
    display(Markdown("### FFT analysis:"))
    plot_filter_fft(x, y, fs, f"White Noise Notch (rho 0.9, alpha {alpha}) (Float 32 KIT)")

## *CHIRP* Sweep $0Hz-20kHz$ filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.9 e alpha variavel

In [ ]:
for alpha in [0, 5, 10, 20]:

    display(Markdown(f"### *CHIRP* filtrado utilizando o kit de desenvolvimento com STM32 (Float 32) filtro Notch rho 0.9 e alpha {alpha}"))
    # Sweep containing all frequencies from 0 Hz to 20 kHz
    fs = 48000
    t = np.linspace(0, 10, fs * 10)
    x = signal.chirp(t, 0, 10, 20000, method="linear")

    # Read audio file
    fs, y = wavfile.read(f"./generated/kit/notch/f32/rho_0.9/alpha_{alpha}/chirp-filtered.wav")

    # Allow play filtered audio
    display(Markdown("### Original audio:"))
    display(ipd.Audio(x, rate=fs))

    # Allow play filtered audio
    display(Markdown("### Filtered audio:"))
    display(ipd.Audio(y, rate=fs))

    # Plot FFT analysis using the new function
    display(Markdown("### FFT analysis:"))
    plot_filter_fft(x, y, fs, f"*CHIRP* Notch (rho 0.9, alpha {alpha}) (Float 32 KIT)")

# 9) Comparação de performance entre os filtros

Para o calculo de performance dos filtros foi utilizada a tecnica *GPIO Toggle Profiling* que consiste em verificar o tempo de resposta de chaveamento de um pino de GPIO e depois mudar o estado do pino no inicio da execução e no final da execução.

Primeiramente foi necessario fazer o profile da HAL do STM32 para verificar o tempo de resposta.

![Hal_Profile](./assets/perf/hal_interference.png)

A partir de essa medição foi assumido que o tempo de resposta de toggle do pino e de 239.37ns.

## Tempo de execução do filtro FIR Float 32

![Profile FIR Float 32](./assets/perf/fir/f32/fir_f32_period.png)
![Profile FIR Float 32](./assets/perf/fir/f32/fir_f32_width.png)

O tempo de execução do filtro FIR Float 32 com 205 coeficientes é de $ 17.245\text{ ms} - 239.37\text{ ns} = 17.244761\text{ ms}$
O período de execução é de $ 21.32\text{ ms}$

## Tempo de execução do filtro FIR Q15

![Profile FIR Q15](./assets/perf/fir/q15/fir_q15_period.png)
![Profile FIR Q15](./assets/perf/fir/q15/fir_q15_width.png)

O tempo de execução do filtro FIR Q15 com 205 coeficientes é de $ 16.424\text{ ms} - 239.37\text{ ns} = 16.423761\text{ ms}$
O período de execução é de $ 21.32\text{ ms}$

## Tempo de execução do filtro IIR Float 32

![Profile IIR Float 32](./assets/perf/iir/f32/iir_f32_period.png)
![Profile IIR Float 32](./assets/perf/iir/f32/iir_f32_width.png)

O tempo de execução do filtro IIR Float 32 com 9 coeficientes é de $ 1.0503\text{ ms} - 239.37\text{ ns} = 1.050061\text{ ms}$
O período de execução é de $ 21.31\text{ ms}$

Com os resultados obtidos, foi possível observar que o filtro FIR Q15 e Float 32 não possuem tanta diferença de desempenho comparado com o filtro IIR, sendo aproximadamente 16 vezes mais lento que o filtro IIR.
A performance é consistente com o observado nos tópicos anteriores.

## Tempo de execução do filtro Notch Float 32

![Profile Notch Float 32](./assets/perf/notch/f32/notch_f32_period.png)
![Profile Notch Float 32](./assets/perf/notch/f32/notch_f32_width.png)

O tempo de execução do filtro Notch Float 32 com 6 coeficientes é de $ 555.65\text{ µs} - 239.37\text{ ns} = 555.410630\text{ µs}$
O período de execução é de $ 21.31\text{ ms}$


O filtro Notch tem a maior seletividade se bem configurado ja que pequenas variações na frequencia do tom de ruido podem fazer que saia da janela de rejeição.
Mas e o filtro que usa menos coeficientes com muito boa performance.




In [ ]:
def create_filter_comparison_table_audio():
    """
    Create a comparative table only with filters applied to Audio signal
    Sorted by gain at target frequency (best attenuation first)
    Shows only the most relevant metrics for audio filtering
    """
    if not filters_analysis_data:
        return "### No filters analyzed yet!"

    # Filter only filters that contain "Audio" or "Tone" in the name
    audio_filters = {name: data for name, data in filters_analysis_data.items()
                    if "Audio" in name or "Tone" in name or "audio" in name}

    if not audio_filters:
        return "### No Audio filters analyzed yet!"

    # Sort by gain at target frequency (more negative = better attenuation)
    sorted_filters = sorted(
        audio_filters.items(),
        key=lambda x: x[1].target_freq_gain_db if x[1].target_freq_gain_db is not None else 0
    )

    # Header
    table = "## Filter Metrics Comparison (Audio File with Tone)\n"
    table += "*Sorted by gain at target frequency (best attenuation first)*\n\n"
    table += "| Filter | Gain @ 875Hz (dB) | 3dB Bandwidth (Hz) |\n"
    table += "|--------|-------------------|-----------------|\n"

    # Rows - one filter per line
    for filter_name, data in sorted_filters:
        target_gain = f"{data.target_freq_gain_db:.2f}" if data.target_freq_gain_db is not None else "N/A"
        bandwidth = f"{data.bandwidth_3db:.2f}" if data.bandwidth_3db is not None else "N/A"

        table += f"| {filter_name} | {target_gain} | {bandwidth} |\n"

    return table


# Execute and display tables
comparison_table_chirp = create_filter_comparison_table_chirp()
display(Markdown(comparison_table_chirp))

comparison_table_audio = create_filter_comparison_table_audio()
display(Markdown(comparison_table_audio))


# Conclusão


Os filtros FIR (Float 32 e Q15) apresentaram a melhor atenuação no áudio real (aproximadamente -29 dB na frequência de 875 Hz), porém consomem 17.24 ms de processamento, representando cerca de 77-81% do período disponível devido aos 205 coeficientes necessários. 
O filtro IIR mostrou-se significativamente mais eficiente, executando em apenas 1.05 ms com apenas 9 coeficientes, mantendo uma atenuação respeitável de -18 dB. 
O filtro Notch destacou-se como a solução mais eficiente computacionalmente, executando em apenas 555.41 µs com apenas 6 coeficientes, sendo 47 vezes mais rápido que FIR e 2 vezes mais rápido que IIR, com atenuação entre-11 a -18 dB dependendo da configuração.
Para aplicações de tempo real em sistemas embarcados, o filtro Notch ou IIR representam as melhores escolhas para este caso de rejeição de um tom de unica frequencia.